安装opencompass：Kaggle上已经为我们准备好了其他常用包，只需安装opencompass用于评测即可。如果不在Kaggle上运行，则还需要安装其他必要包。

In [ ]:
!pip install "opencompass[full]"
# !pip install torch transformers datasets "opencompass[full]"


In [7]:
import kagglehub

# Download latest version
path = kagglehub.model_download("qwen-lm/qwen2.5/transformers/0.5b")

print("Path to model files:", path)


100%|██████████| 11.1k/11.1k [00:00<00:00, 19.3MB/s]

  0%|          | 0.00/1.48k [00:00<?, ?B/s]

100%|██████████| 1.48k/1.48k [00:00<00:00, 759kB/s]





100%|██████████| 681/681 [00:00<00:00, 385kB/s]
100%|██████████| 3.76k/3.76k [00:00<00:00, 659kB/s]
100%|██████████| 138/138 [00:00<00:00, 36.3kB/s]







100%|██████████| 1.59M/1.59M [00:01<00:00, 1.05MB/s]




100%|██████████| 7.06k/7.06k [00:00<00:00, 3.04MB/s]





100%|██████████| 6.71M/6.71M [00:02<00:00, 2.51MB/s]







100%|██████████| 2.65M/2.65M [00:01<00:00, 1.61MB/s]




















































































































































































































































































100%|██████████| 942M/942M [01:37<00:00, 10.2MB/s]

Path to model files: /home/xiaxinyuan/.cache/kagglehub/models/qwen-lm/qwen2.5/transformers/0.5b/1


In [9]:

# Download latest version
path = kagglehub.model_download("qwen-lm/qwen2.5/transformers/3b")

print("Path to model files:", path)

100%|██████████| 138/138 [00:00<00:00, 271kB/s]


100%|██████████| 1.48k/1.48k [00:00<00:00, 633kB/s]

  0%|          | 0.00/3.75k [00:00<?, ?B/s]




100%|██████████| 7.21k/7.21k [00:00<00:00, 2.62MB/s]
100%|██████████| 3.75k/3.75k [00:00<00:00, 395kB/s]
100%|██████████| 683/683 [00:00<00:00, 67.8kB/s]




100%|██████████| 7.06k/7.06k [00:00<00:00, 7.45MB/s]

















100%|██████████| 34.7k/34.7k [00:00<00:00, 240kB/s]


100%|██████████| 1.59M/1.59M [00:02<00:00, 755kB/s]















































100%|██████████| 2.65M/2.65M [00:01<00:00, 1.50MB/s]
















100%|██████████| 6.71M/6.71M [00:02<00:00, 2.65MB/s]










































































































































































































































































































































































































































































































































































































































































































































Path to model files: /home/xiaxinyuan/.cache/kagglehub/models/qwen-lm/qwen2.5/transformers/3b/1


In [8]:
alpaca_language_instruction_training_path = kagglehub.dataset_download('thedevastator/alpaca-language-instruction-training')

print("Path to dataset files:", alpaca_language_instruction_training_path)

100%|██████████| 13.9M/13.9M [00:03<00:00, 4.34MB/s]

Extracting files...


Path to dataset files: /home/xiaxinyuan/.cache/kagglehub/datasets/thedevastator/alpaca-language-instruction-training/versions/2


In [4]:
import wandb
with open('wandb.txt', 'r') as file:
    wandb_key = file.read()
wandb.login(key=wandb_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: char-lotte (char-lotte-xia). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/xiaxinyuan/.netrc


True

# 指令微调

In [1]:
"""
The main program for finetuning LLMs with Huggingface Transformers Library.

ALL SECTIONS WHERE CODE POSSIBLY NEEDS TO BE FILLED IN ARE MARKED AS TODO.
"""
# !pip install --upgrade Pillow
import argparse
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import sys
import torch
from transformers import TrainingArguments, HfArgumentParser, Trainer, AutoTokenizer, AutoModelForCausalLM
import datasets

/home/xiaxinyuan/.conda/envs/dino/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Define the arguments required for the main program.
# NOTE: You can customize any arguments you need to pass in.
@dataclass
class ModelArguments:
    """Arguments for model
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the LLM to fine-tune or its name on the Hugging Face Hub."
        }
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype."
            ),
            "choices": ["bfloat16", "float16", "float32"],
        },
    )
    # TODO: add your model arguments here
    pass


@dataclass
class DataArguments:
    """Arguments for data
    """
    dataset_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the fine-tuning dataset or its name on the Hugging Face Hub."
        }
    )
    # TODO: add your data arguments here
    max_length: int = field(
        default=512,
        metadata={
            "help": "The max length of tokenized data."
        }
    )
    skip_too_long: Optional[bool] = field(
        default = False,
        metadata = {
            "help" : "whether to skip those longer than max length of tokenized data"
        }
    )

In [13]:
# The main function
# NOTE You can customize some logs to monitor your program.
def finetune():
    # TODO Step 1: Define an arguments parser and parse the arguments
    # NOTE Three parts: model arguments, data arguments, and training arguments
    # HINT: Refer to 
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/internal/trainer_utils#transformers.HfArgumentParser
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments
    # parser = ...
    # model_args, data_args, training_args = ...
    ### cjy
    parser = HfArgumentParser(dataclass_types=[ModelArguments, DataArguments, TrainingArguments])
    """
    hugging face argument parser
    """
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    """
    return tuple
    DataClass 实例的顺序与传递给 initializer.abspath 的顺序相同
    如果适用，将更多（非 DataClass 支持的）参数的附加命名空间添加到解析器 初始化后。
    剩余参数字符串的可能列表。（与 argparse 相同。ArgumentParser.parse_known_args）
    """
    ### cjy

    # TODO Step 2: Load tokenizer and model
    # HINT 1: Refer to
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/tokenizer#tokenizer
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/qwen2
    # HINT 2: To save training GPU memory, you need to set the model's parameter precision to half-precision (float16 or bfloat16).
    #         You may also check other strategies to save the memory!
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/llama2#usage-tips
    #   * https://huggingface.co/docs/transformers/perf_train_gpu_one
    #   * https://www.53ai.com/news/qianyanjishu/2024052494875.html
    # tokenizer = ...
    # model = ...
    ## cjy
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_args.model_name_or_path)
    """
    从预训练模型的 tokenizer 配置中加载 tokenizer 的配置
    input:
        pretrained_model_name_or_path: 预训练模型的名称或路径（tokenizer 配置文件 所在的目录 即“Qwen2.5-0.5B”）
    功能:
        1 函数首先尝试从指定的模型名称或路径加载 tokenizer 配置文件
        2 如果找不到配置文件，则从模型库中加载默认的 tokenizer 配置
    """
    model = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=model_args.model_name_or_path, 
        torch_dtype=model_args.torch_dtype,
        trust_remote_code=True,  # Qwen模型需要这个参数
        device_map="auto"  # 可选，用于自动处理模型加载到设备
    )
    """
    用于从预训练模型加载模型实例。 
    """
    ### cjy

    # TODO Step 3: Load dataset
    # HINT: https://huggingface.co/docs/datasets/v3.1.0/en/package_reference/main_classes#datasets.Dataset
    # dataset = ...
    ### cjy
    dataset = datasets.load_dataset(path='csv', data_files=data_args.dataset_path)
    ### cjy

    # TODO Step 4: Define the data collator function
    # NOTE During training, for each model parameter update, we fetch a batch of data, perform a forward and backward pass,
    # and then update the model parameters. The role of the data collator is to process the data (e.g., padding the data within
    # a batch to the same length) and format the batch into the input required by the model.
    # 数据整理器（data collator）函数。这个函数的主要任务是处理从数据集中加载的每个数据批次，使其符合模型所需的输入格式
    # In this assignment, the purpose of the custom data_collator is to process each batch of data from the dataset loaded in
    # Step 3 into the format required by the model. This includes tasks such as tokenizing the data, converting each token into 
    # an ID sequence, applying padding, and preparing labels.
    # 对数据进行标记化（tokenization）。将每个标记转换为 ID 序列。填充（padding），确保批次中的所有数据具有相同的长度。准备标签（labels），以便在训练过程中使用
    # HINT:
    #   * Before implementation, you should:
    #      1. Clearly understand the format of each sample in the dataset loaded in Step 3.
    #      2. Understand the input format required by the model (https://huggingface.co/docs/transformers/model_doc/qwen2#transformers.Qwen2ForCausalLM).
    #         Reading its source code also helps!

    def data_collator(batch: List[Dict]):
        """
        batch: list of dict, each dict of the list is a sample in the dataset.
        """
        # pass
        ### cjy
        # 期望字典输出
        new_batch = {
            "input_ids": [], #  (torch.LongTensor of shape (batch_size, sequence_length)) 
            # input ids: 通常是作为输入传递给模型的唯一必需参数. token(模型输入)的数字化表示. https://huggingface.co/docs/transformers/glossary#input-ids
            # 用 PreTrainedTokenizer.encode() 方法得到
            "labels": [], #  (torch.LongTensor of shape (batch_size, sequence_length))
            # labels: 标签是模型训练的目标值，用于计算损失函数。 (loss, optional, returned when labels is provided))
            # 用 PreTrainedTokenizer.encode() 方法得到 
            "attention_mask": [],
            # 将不同长度的token摆在一起，pad的部分设为0
            "num_logits_to_keep" : 0,

        }
        ## cyd
        inputs = []
        labels = []
        max_length = 0

        for sample in batch:
            instruction = sample.get("instruction","")
            input_text = sample.get("input","")
            output_text = sample.get("output","")

            # 检查input和output是否为空
            # if not input_text.strip():
            #     input_text = f"\n{input_text}"

            if not output_text.strip():
                output_text = "<empty>"
            
            # 构建输入序列
            input_ids = tokenizer.encode_plus(
                f"{instruction}. The input is {input_text}", # 将instruction和input_text进行拼接，生成文本输入
                return_tensors = "pt", # 输出转换为pytorch的张量格式
                max_length = tokenizer.model_max_length, # 如果输入序列超过最大长度则截断
                truncation = True, 
                padding = False, # 即使输入序列没有达到最大长度，也不进行填充
            ).input_ids # 用于获取tokenizer返回字典中的‘input_ids’字段
            if input_ids[0, -1] == tokenizer.eos_token_id:
                input_ids = input_ids[:, :-1]
            # 构建输出序列
            output_ids = tokenizer.encode_plus(
                output_text,
                return_tensors = "pt",
                max_length = tokenizer.model_max_length,
                truncation = True,
                padding = False,
            ).input_ids
            if input_ids[0, -1] != tokenizer.eos_token_id:
                input_ids = torch.cat([input_ids, torch.tensor([[tokenizer.eos_token_id]], device=input_ids.device)], dim=1)
            # 拼接输入与输出序列，获得模型所需的input_ids
            full_input = torch.cat([input_ids, output_ids], dim=1)
            # if (full_input.size(1)>data_args.max_length):
            #     continue
            inputs.append(full_input)

            # 创建标签张量
            labels_tensor = torch.full_like(full_input, -100) # 用-100填充表示这些位置在损失计算中被忽略
            labels_tensor[:, input_ids.shape[1]:] = full_input[:, input_ids.shape[1]:] # 将output_ids对应的位置替换为output_ids原来的值，input_ids对应的位置仍为-100，表示学习时只学习output部分
            labels.append(labels_tensor)

            if full_input.shape[1] > max_length:
                max_length = full_input.shape[1]


        # 处理batch的padding，将同一个batch的label和input都填充到同一个长度
        # inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=tokenizer.pad_token_id)
        # labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)
        inputs = [torch.nn.functional.pad(input, (0,max_length - input.size(1)),value=tokenizer.pad_token_id) for input in inputs]
        labels = [torch.nn.functional.pad(label, (0, max_length - label.size(1)), value=-100) for label in labels]

        # inputs = torch.stack(inputs)
        # labels = torch.stack(labels)

        inputs = torch.cat(inputs, dim=0)
        labels = torch.cat(labels, dim=0)

        return {
            "input_ids": inputs,
            "labels": labels,
            "attention_mask": ((inputs != tokenizer.pad_token_id) & (inputs != labels)).to(dtype=torch.int), 
            # pad_token_id是tokenizer定义的填充令牌ID，也就是对padding的部分填充一个特殊的令牌
            # 该行代码将生成一个与inputs张量形状相同的布尔张量，其中值为True:表示对应的输入ID不是填充ID（即该令牌是有效的；值为False: 表示对应的输入ID是填充ID（即该令牌是无效的，需要被忽略）。
        }
        ## cyd

            # print("error")
            # max_length=data_args.max_length
            # min_input_length=1e6
            # for sample in batch:
            #     # resume one bit for <eos>
            #     input_id=tokenizer.encode("instruction:"+sample["instruction"] if sample["instruction"] else "" + "\n input:"+ sample["input"] if sample["input"] else "", 
            #                                                    max_length=data_args.max_length-1, truncation=True,padding=False)
            #     output_id=tokenizer.encode("output:"+sample["output"] if sample["output"] else "",
            #                                max_length=tokenizer.model_max_length-1, truncation=True, padding=False)
            #     full_input=input_id+output_id
            #     attention_mask_one=[True for _ in input_id] + [False for _ in output_id]
            #     # max_length=len(full_input) if (len(full_input)>max_length) else max_length
            #     # min_input_length=len(input_id) if (len(input_id)<min_input_length) else min_input_length
                
                
            #     new_batch["input_ids"].append(torch.tensor(full_input).unsqueeze(0))
            #     new_batch["attention_mask"].append(torch.tensor(attention_mask_one).to(dtype=torch.int).unsqueeze(0))
            
            # new_batch["num_logits_to_keep"] = max_length-min_input_length
    
    
            # new_batch["input_ids"] = [torch.nn.functional.pad(input, (0,max_length - input.size(1)),value=tokenizer.pad_token_id) for input in new_batch["input_ids"]]
            # new_batch["attention_mask"]=[torch.nn.functional.pad(mask, (0,max_length - mask.size(1)),value=False) for mask in new_batch["attention_mask"]]
    
            # new_batch["input_ids"] = torch.cat(new_batch["input_ids"],dim=0)
            # new_batch["attention_mask"] = torch.cat(new_batch["attention_mask"],dim=0).to(dtype=torch.int)
            # new_batch["labels"]=new_batch["input_ids"][:,-new_batch["num_logits_to_keep"]:]
    
            # # print(new_batch["input_ids"].shape,new_batch["attention_mask"].shape,new_batch["labels"].shape)
            # return new_batch

    # TODO Step 5: Define the Trainer
    # HINT: https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        # ...,
        # model=model,
        ### cjy
        model=model, # 模型  (PreTrainedModel or torch.nn.Module)
        args=training_args, # 训练参数 (TrainingArguments)
        data_collator=data_collator, # 数据整理器 (DataCollator)
        train_dataset=dataset['train'], # 训练数据集 (Dataset)
        tokenizer=tokenizer,
        ### cjy
    )
    # Step 6: Train!
    trainer.train()

In [ ]:
# import pandas as pd
# # test min_input_length:
# token_lens = []
# input_file="/kaggle/input/alpaca-language-instruction-training/train.csv"  # 训练集文件
# # 读取CSV文件
# df = pd.read_csv(input_file)

# # 将DataFrame转换为所需的格式
# infer_data = []
# for _, row in df.iterrows():
#     # 假设CSV文件中的列名为'instruction', 'input', 'output'
#     # 如果input为空，则只使用instruction作为query
#     query = row['instruction']
#     if pd.notna(row['input']) and row['input'].strip():  # 检查input是否存在且非空
#         query = query + "\n" + row['input']
    
#     item = {
#         'query': query,
#         'answer': row['output'] if pd.notna(row['output']) else ''  # 处理可能的空值
#     }
#     infer_data.append(item)

# # 验证数据格式
# print(f"数据集大小: {len(infer_data)}")
# print("\n示例数据:")
# print(infer_data[0])

In [ ]:
# import pandas as pd
# from tqdm.auto import tqdm as tqdm_auto  # 修改导入方式

# def calculate_token_lengths(tokenizer):
#     token_lens = []
#     max_length = 0
    
#     # 读取数据
#     input_file = "/kaggle/input/alpaca-language-instruction-training/train.csv"
#     df = pd.read_csv(input_file)
    
#     print("开始计算token长度...")
#     for index in list(tqdm_auto(range(len(df)))):
#         sample = df.iloc[index]
#         input_id = tokenizer.encode(
#             "instruction:" + (sample["instruction"] if pd.notna(sample["instruction"]) else "") + 
#             "\n input:" + (sample["input"] if pd.notna(sample["input"]) else ""),
#             max_length=tokenizer.model_max_length,
#             truncation=True,
#             padding=False
#         )
        
#         # 构造输出序列
#         output_id = tokenizer.encode(
#             "output:" + (sample["output"] if pd.notna(sample["output"]) else ""),
#             max_length=tokenizer.model_max_length,
#             truncation=True,
#             padding=False
#         )
        
#         # 计算完整序列长度
#         full_length = len(input_id) + len(output_id)
#         token_lens.append(full_length)
        
#         if full_length > max_length:
#             max_length = full_length
#             max_sample = {
#                 'instruction': sample["instruction"],
#                 'input': sample["input"],
#                 'output': sample["output"],
#                 'length': full_length
#             }
    
#     print(f"\n统计信息:")
#     print(f"最大长度: {max(token_lens)}")
#     print(f"最小长度: {min(token_lens)}")
#     print(f"平均长度: {sum(token_lens)/len(token_lens):.2f}")
#     print(f"中位数长度: {sorted(token_lens)[len(token_lens)//2]}")
    
#     print(f"\n最长样本信息:")
#     print(f"长度: {max_sample['length']}")
#     print(f"Instruction: {max_sample['instruction'][:100]}...")
#     print(f"Input: {max_sample['input'][:100] if pd.notna(max_sample['input']) else 'None'}...")
#     print(f"Output: {max_sample['output'][:100]}...")
    
#     # 绘制长度分布直方图
#     import matplotlib.pyplot as plt
#     plt.figure(figsize=(10, 6))
#     plt.hist(token_lens, bins=50)
#     plt.title('Token Length Distribution')
#     plt.xlabel('Token Length')
#     plt.ylabel('Count')
#     plt.show()

# tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="/kaggle/input/qwen2.5/transformers/0.5b/1")
# calculate_token_lengths(tokenizer)

In [14]:
# Pass your training arguments.
# NOTE [IMPORTANT!!!] DO NOT FORGET TO PASS PROPER ARGUMENTS TO SAVE YOUR CHECKPOINTS!!!
sys.argv = [
    "notebook", 
    # "--arg1", "value1",
    # "--arg2", "value2",
    # ...
    ### cjy
    "--model_name_or_path", "/home/xiaxinyuan/.cache/kagglehub/models/qwen-lm/qwen2.5/transformers/0.5b/1/",
    "--dataset_path", "/home/xiaxinyuan/.cache/kagglehub/datasets/thedevastator/alpaca-language-instruction-training/versions/2/train.csv",
    "--torch_dtype", "float32", #see Qwen2.5-0.5B/config.json?
    "--output_dir", "/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/output/", # --output_dir 参数在 TrainingArguments 中有
    "--remove_unused_columns", "False", #ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [output, instruction, input]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`
    "--max_length", "512",
    ### xxy
    "--skip_too_long","True",
    "--learning_rate","1e-5",
    "--lr_scheduler_type","cosine",
    "--optim", "adamw_hf",
    "--warmup_ratio", "0.03",
    "--weight_decay", "0.003",
    ### xxy
    ### cjy
    "--per_device_train_batch_size", "4",  # 设置训练的 batch size
    "--per_device_eval_batch_size", "4", 
    "--save_steps", "1000",
    "--save_total_limit", "3",
    "--num_train_epochs", "3",  # 通常3-5个epoch即可收敛，长时间训练可能会过拟合 
    "--fp16","True",  # 开启混合精度加速
    "--overwrite_output_dir","True",  #开发过程中覆盖旧的文件

    
]
finetune()
# bug: TypeError: 'list' object is not a mapping 通常是因为在训练过程中，模型期望接收到一个字典格式的输入，但实际上接收到了一个列表。问题可能出现在 data_collator 函数的返回值上。

/tmp/ipykernel_1034862/2375760050.py:204: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/xiaxinyuan/.conda/envs/dino/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.13 GiB. GPU 0 has a total capacity of 23.64 GiB of which 501.38 MiB is free. Process 2157232 has 2.59 GiB memory in use. Including non-PyTorch memory, this process has 20.53 GiB memory in use. Of the allocated memory 18.72 GiB is allocated by PyTorch, and 1.35 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 

In [ ]:
!nvidia-smi

# 评测模型

In [ ]:
PLM_MODEL_PATH = "/kaggle/input/qwen2.5/transformers/0.5b/1"
SFT_MODEL_PATH = "/kaggle/working/output/checkpoint-20000"

In [ ]:
! ls /kaggle/working/

如果你有多个GPU，可以修改下面的--hf-num-gpus参数来加速评测。

In [ ]:
# !opencompass \
#     --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
#     --summarizer example \
#     --hf-type base \
#     --hf-path {PLM_MODEL_PATH} \
#     --tokenizer-kwargs padding_side="left" truncation="left" \
#     --max-seq-len 2048 \
#     --batch-size 4 \
#     --hf-num-gpus 2 \
#     --work-dir "/kaggle/working/evals/plm" \
#     --debug


In [ ]:
opencompass \
    --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path 'output/1430/checkpoint-50000' \
    --tokenizer-kwargs padding_side="left" truncation="left" \
    --max-seq-len 2048 \
    --batch-size 4 \
    --hf-num-gpus 8 \
    --work-dir "/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/evals/plm" \
    --debug

In [ ]:
# !cd /
# !opencompass \
#     --datasets mmlu_ppl \
#     --summarizer example \
#     --hf-type base \
#     --hf-path {SFT_MODEL_PATH} \
#     --tokenizer-kwargs padding_side="left" truncation="left" \
#     --max-seq-len 2048 \
#     --batch-size 4 \
#     --hf-num-gpus 2 \
#     --work-dir "/kaggle/working/evals/sft" \
#     --debug


# Calculate similarity between datasets on finetuned model

